In [1]:
# Define a convolution neural network
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*10*10, 10)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.pool(output)
        output = F.relu(self.bn4(self.conv4(output)))
        output = F.relu(self.bn5(self.conv5(output)))
        print("Dimensiones de output antes de cambiar la forma:", output.view())
        output = output.view(4, 24, 634, 954)
        output = self.fc1(output)

        return output

NameError: name 'nn' is not defined

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from torchvision import datasets
import torchvision
import torchvision.transforms as transforms

test_loader = DataLoader(
    datasets.ImageFolder("drive/MyDrive/Dataset/test",
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ]),
    ),
    batch_size=4,
    shuffle=True,
)
# Carga tu modelo entrenado
model = Network()
path = "./drive/MyDrive/myFirstModel.pth"
model.load_state_dict(torch.load(path))
model.eval()

# Define una lista para almacenar las etiquetas reales y las predichas
true_labels = []
predicted_labels = []

# Pasa por el conjunto de prueba y almacena las etiquetas reales y predichas
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        true_labels.extend(labels.numpy())
        predicted_labels.extend(predicted.numpy())

# Convierte las listas en arrays de NumPy para usar en la matriz de confusión
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calcula la matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Imprime la matriz de confusión
print("Matriz de Confusión:")
print(conf_matrix)

# También puedes imprimir un informe de clasificación que incluye precision, recall, y f1-score
class_names = ["Clase 0", "Clase 1", "Clase 2", "Clase 3"]
print("\nInforme de Clasificación:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))
